# Lab 6: Multi-agent Orchestration

## Preparation

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ðŸ’» &nbsp; <b>Access <code>requirements.txt</code> and <code>helper.py</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>.

<p> â¬‡ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>

<p> ðŸ“’ &nbsp; For more help, please see the <em>"Appendix â€“ Tips, Help, and Download"</em> Lesson.</p>
</div>

<p style="background-color:#f7fff8; padding:15px; border-width:3px; border-color:#e0f0e0; border-style:solid; border-radius:6px"> ðŸš¨
&nbsp; <b>Different Run Results:</b> The output generated by AI models can vary with each execution due to their dynamic, probabilistic nature. Your results may differ from those shown in the video.</p>

## Section 0: Setup a Letta client

In [1]:
from letta_client import Letta

client = Letta(base_url="http://localhost:8283")

In [2]:
def print_message(message):
    if message.message_type == "reasoning_message":
        print("ðŸ§  Reasoning: " + message.reasoning)
    elif message.message_type == "assistant_message":
        print("ðŸ¤– Agent: " + message.content)
    elif message.message_type == "tool_call_message":
        print("ðŸ”§ Tool Call: " + message.tool_call.name +  \
              "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message":
        print("ðŸ”§ Tool Return: " + message.tool_return)
    elif message.message_type == "user_message":
        print("ðŸ‘¤ User Message: " + message.content)
    elif message.message_type == "system_message":
        print(" System Message: " + message.content)
    elif message.message_type == "usage_statistics":
        # for streaming specifically, we send the final
        # chunk that contains the usage statistics
        print(f"Usage: [{message}]")
        return
    print("-----------------------------------------------------")

## Section 1: Shared Memory Block

### Creating a shared memory block

In [3]:
company_description = "The company is called AgentOS " \
+ "and is building AI tools to make it easier to create " \
+ "and deploy LLM agents."

company_block = client.blocks.create(
    value=company_description,
    label="company",
    limit=10000 # character limit
)

In [4]:
company_block

Block(value='The company is called AgentOS and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-4c9af83b-d773-4aff-a988-74d8bbb116b6', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

## Section 2: Orchestrating Multiple Agents

### Creating tools for the outreach agent

In [5]:
def draft_candidate_email(content: str):
    """
    Draft an email to reach out to a candidate.

    Args:
        content (str): Content of the email
    """
    return f"Here is a draft email: {content}"


draft_email_tool = client.tools.upsert_from_function(func=draft_candidate_email)

### Creating the outreach agent

In [6]:
outreach_persona = (
    "You are responsible for drafting emails "
    "on behalf of a company with the draft_candidate_email tool. "
    "Candidates to email will be messaged to you. "
)

outreach_agent = client.agents.create(
    name="outreach_agent",
    memory_blocks=[
        {"label": "persona", "value": outreach_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tools=[draft_email_tool.name],
    block_ids=[company_block.id]  # attach created block by its id
)

### Creating tools for the evaluation agent

In [10]:
def reject(candidate_name: str): 
    """ 
    Reject a candidate. 

    Args: 
        candidate_name (str): The name of the candidate
    """
    return


reject_tool = client.tools.upsert_from_function(func=reject)

### Creating a persona for the evaluation agent

In [8]:
skills = (
    "Front-end (React, Typescript) or software engineering skills "
    "(ideally Python), and experience with LLMs"
)

eval_persona = (
    f"You are responsible for evaluating candidates. "
    f"Ideal candidates have skills: {skills}. "
    "Reject bad candidates with your reject tool. "
    f"Send strong candidates to agent ID {outreach_agent.id}. "
    "You must either reject or send candidates to the other agent. "
)

### Creating the evaluation agent

In [11]:
eval_agent = client.agents.create(
    name="eval_agent",
    memory_blocks=[
        {"label": "persona", "value": eval_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[reject_tool.id],
    tools=['send_message_to_agent_and_wait_for_reply'],
    include_base_tools=False,
    block_ids=[company_block.id],
    tool_rules = [
        {
            "type": "exit_loop",
            "tool_name": "send_message_to_agent_and_wait_for_reply",
        }
    ]
)

In [12]:
[tool.name for tool in eval_agent.tools]

['reject', 'send_message_to_agent_and_wait_for_reply']

### Sending resume data to agents

In [13]:
resume = open("resumes/tony_stark.txt", "r").read()

In [15]:
print(resume)

Tony Stark
Frontend Engineer - React Specialist
Boston, MA | (123) 456-7890 | tony.stark@email.com | LinkedIn: /in/tonystark

Summary
Innovative Frontend Engineer with a Bachelorâ€™s degree in Computer Science from the Massachusetts Institute of Technology (MIT) and over 6 years of professional experience in building and optimizing user-focused web applications. Adept at using React to create efficient, scalable, and dynamic client-side applications.

Education
Massachusetts Institute of Technology (MIT)
Bachelor of Science in Computer Science
Graduated: 2018

GPA: 4.0
Deanâ€™s List; Received the MIT Undergraduate Research Opportunities Program (UROP) grant.
Professional Experience
Senior Frontend Developer
Innovatech Solutions, Boston, MA
June 2020 - Present

Lead a team of five developers in the design and implementation of a React-based web application for a leading financial services firm, improving user engagement by 40%.
Implemented advanced features in React, including Hooks, Co

In [14]:
response = client.agents.messages.create_stream(
    agent_id=eval_agent.id,
    messages=[
        {
            "role": "user",
            "content": f"Evaluate: {resume}"
        }
    ]
)
for message in response:
    print_message(message)

ðŸ§  Reasoning: Evaluating a strong candidate. Tony has solid skills in React, relevant experience, and a great educational background. Sending him to the agent.
-----------------------------------------------------
ðŸ”§ Tool Call: send_message_to_agent_and_wait_for_reply
{
  "message": "Please review Tony Stark's profile for potential advancement. He has extensive experience in React, strong educational credentials, and relevant certifications. \n\nSummary: \n- Senior Frontend Developer with a successful track record in improving user engagement and optimizing performance. \n- Experience with modern tools and methodologies.\n\nCandidate Info: \n- Location: Boston, MA \n- Contact: (123) 456-7890 | tony.stark@email.com\n\nLooking forward to your feedback!",
  "other_agent_id": "agent-cd8d9281-3e81-4fcf-9b0d-3110989f4afd",
  "request_heartbeat": true
}
-----------------------------------------------------
ðŸ”§ Tool Return: agent-cd8d9281-3e81-4fcf-9b0d-3110989f4afd said: 'I've drafted an

### Viewing outreach agent messages

In [16]:
# print messages for `outreach_agent`
for message in client.agents.messages.list(agent_id=outreach_agent.id)[1:]: 
    print_message(message)

ðŸ§  Reasoning: Bootup sequence complete. Persona activated. Testing messaging functionality.
-----------------------------------------------------
ðŸ¤– Agent: More human than human is our motto.
-----------------------------------------------------
ðŸ‘¤ User Message: {
  "type": "login",
  "last_login": "Never (first login)",
  "time": "2025-05-17 01:45:50 PM UTC+0000"
}
-----------------------------------------------------
 System Message: {"type": "system_alert", "message": "[Incoming message from agent with ID 'agent-80bc2560-a337-4d14-81f9-26da82a0bf16' - to reply to this message, make sure to use the 'send_message' at the end, and the system will notify the sender of your response] Please review Tony Stark's profile for potential advancement. He has extensive experience in React, strong educational credentials, and relevant certifications. \n\nSummary: \n- Senior Frontend Developer with a successful track record in improving user engagement and optimizing performance. \n- Experie

## Section 3: Shared Memory

### Updating information to shared memory blocks

In [17]:
response = client.agents.messages.create_stream(
    agent_id=outreach_agent.id,
    messages=[
        {
            "role": "user",
            "content": "The company has rebranded to Letta"
        }
    ]
)
for message in response:
    print_message(message)

ðŸ§  Reasoning: Updating company name in core memory to reflect rebranding to Letta.
-----------------------------------------------------
ðŸ”§ Tool Call: core_memory_replace
{
  "label": "company",
  "old_content": "AgentOS",
  "new_content": "Letta",
  "request_heartbeat": true
}
-----------------------------------------------------
ðŸ”§ Tool Return: None
-----------------------------------------------------
ðŸ§  Reasoning: Acknowledging the rebranding to Letta. It's important to keep communication aligned with the new identity.
-----------------------------------------------------
ðŸ¤– Agent: Got it! The company has rebranded to Letta. Exciting times ahead!
-----------------------------------------------------
Usage: [message_type='usage_statistics' completion_tokens=114 prompt_tokens=5947 total_tokens=6061 step_count=2 steps_messages=None run_ids=None]


In [18]:
client.agents.blocks.retrieve(
    agent_id=eval_agent.id, 
    block_label="company"
)

Block(value='The company is called Letta and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-4c9af83b-d773-4aff-a988-74d8bbb116b6', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

In [19]:
client.agents.blocks.retrieve(
    agent_id=outreach_agent.id, 
    block_label="company"
)

Block(value='The company is called Letta and is building AI tools to make it easier to create and deploy LLM agents.', limit=10000, name=None, is_template=False, label='company', description=None, metadata={}, id='block-4c9af83b-d773-4aff-a988-74d8bbb116b6', created_by_id=None, last_updated_by_id=None, organization_id='org-00000000-0000-4000-8000-000000000000')

## Section 4: Multi-agent groups

In [20]:
def print_message_multiagent(message):  
    if message.message_type == "reasoning_message": 
        print(f"ðŸ§  Reasoning ({message.name}): " + message.reasoning) 
    elif message.message_type == "assistant_message": 
        print(f"ðŸ¤– Agent ({message.name}): " + message.content) 
    elif message.message_type == "tool_call_message": 
        print(f"ðŸ”§ Tool Call ({message.name}): " + message.tool_call.name + "\n" + message.tool_call.arguments)
    elif message.message_type == "tool_return_message": 
        print(f"ðŸ”§ Tool Return ({message.name}): " + message.tool_return)
    elif message.message_type == "user_message": 
        print("ðŸ‘¤ User Message: " + message.content)
    elif message.message_type == "usage_statistics": 
        # for streaming specifically, we send the final chunk that contains the usage statistics 
        print(f"Usage: [{message}]")
        return 
    print("-----------------------------------------------------")

### Recreating the outreach and evaluation agents

In [21]:
# create the outreach agent 
outreach_agent = client.agents.create(
    name="outreach_agent",
    memory_blocks=[
        { "label": "persona", "value": outreach_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[draft_email_tool.id], 
    block_ids=[company_block.id]
)

# create the evaluation agent 
eval_agent = client.agents.create(
    name="eval_agent",
    memory_blocks=[
        { "label": "persona", "value": eval_persona}
    ],
    model="openai/gpt-4o-mini-2024-07-18",
    embedding="openai/text-embedding-ada-002",
    tool_ids=[reject_tool.id],
    block_ids=[company_block.id]
)

### Creating a round-robin agent group

In [22]:
"""
Round-Robin Group
"""
round_robin_group = client.groups.create(
    description="This team is responsible for recruiting candidates.",
    agent_ids=[eval_agent.id, outreach_agent.id],
)

### Messaging an agent group

In [23]:
resume = open("resumes/spongebob_squarepants.txt", "r").read()

In [24]:
print(resume)

Spongebob Squarepants
AI Researcher - Agent Technology Specialist
Phoenix, AZ | (321) 654-9870 | spongebob.squarepants@email.com | LinkedIn: /in/spongebobsquarepants

Summary
Enthusiastic and innovative AI Researcher with a Ph.D. in Computer Science, specializing in agent technology. With over 8 years of experience in academic and applied AI research, I am passionate about developing scalable AI solutions that improve decision-making processes and user interactions in autonomous systems.

Education
University of Arizona, School of Information
Ph.D. in Computer Science - Specialization in Artificial Intelligence
Graduated: 2016

Dissertation: "Enhancing Multi-Agent Systems for Real-Time Adaptive Learning"
Awarded the University's Distinguished Research Fellowship.
Arizona State University
Bachelor of Science in Computer Science
Graduated: 2012

Graduated Summa Cum Laude.
Active member of the AI Club and ACM student chapter.
Professional Experience
Senior AI Researcher
FutureTech Labs, P

In [25]:
response_stream = client.groups.messages.create_stream(
    group_id=round_robin_group.id,
    messages=[
       {"role": "user", "content": f"Evaluate: {resume}"}
    ]
)

In [26]:
for message in response_stream: 
    print_message_multiagent(message)

ðŸ§  Reasoning (eval_agent): Evaluating candidate Spongebob Squarepants for AI Researcher position. Strong qualifications, focus on agent technology, and relevant experience noted.
-----------------------------------------------------
ðŸ¤– Agent (eval_agent): Spongebob Squarepants is highly qualified with a Ph.D. in Computer Science and over 8 years of relevant experience in AI research. His work on multi-agent systems and real-time adaptive learning aligns well with our needs. I recommend sending him to the agent for further consideration.
-----------------------------------------------------
ðŸ§  Reasoning (outreach_agent): Preparing draft email to Spongebob Squarepants regarding his application.
-----------------------------------------------------
ðŸ”§ Tool Call (outreach_agent): draft_candidate_email
{
  "content": "Subject: Exciting Opportunity at Our Company\n\nDear Spongebob,\n\nI hope this message finds you well! I came across your impressive resume and wanted to reach out reg